This file uses the run generated by `scenario_hsi_in_typical_run.py` and floweaver
to produce a Sankey diagram that maps appointments with HSI events.

Below is the instruction to run the file.

### Install floweaver in Anaconda Prompt (if use Jupyter Notebook) / PyCharm Terminal:

pip install floweaver

pip install ipysankeywidget

jupyter nbextension enable --py --sys-prefix ipysankeywidget

jupyter notebook (to open jupyter notebook)

### To display and save the output figures:
Select Start Jupyter Server from the Jupyter Actions Menu (lightbulb icon next to Run All cells icon)

Open Event Log

Open in Browser

Find the script and run all cells


In [1]:
import tlo

from pathlib import Path

from tlo.analysis.utils import get_scenario_outputs, load_pickled_dataframes

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt

from ipysankeywidget import SankeyWidget

from floweaver import *

In [2]:
# Declare the name of the file that specified the scenarios used in this run.
scenario_filename = 'scenario_hsi_in_typical_run.py'

# Declare usual paths:
# Get the tlo path
tlopath = Path(tlo.__file__).parent.parent.parent
outputspath = tlopath / Path('outputs/bshe@ic.ac.uk')

# Find results folder (most recent run generated using that scenario_filename)
results_folder = get_scenario_outputs(scenario_filename, outputspath)[-1]
print(f"Results folder is: {results_folder}")

# Declare path for output graphs from this script
make_graph_file_name = lambda stub: results_folder / f"{stub}.png"  # noqa: E731

# Extract results
log = load_pickled_dataframes(results_folder)['tlo.methods.healthsystem']  # (There was only one draw and one run)

# Number of HSI:
hsi = log['HSI_Event']
hsi["date"] = pd.to_datetime(hsi["date"])
hsi["month"] = hsi["date"].dt.month
hsi["Module"] = hsi["TREATMENT_ID"].str.split('_').apply(lambda x: x[0])

# Demand for appointments
num_hsi_by_treatment_id = pd.DataFrame(hsi.groupby(hsi.TREATMENT_ID)['Number_By_Appt_Type_Code'].size())
num_hsi_by_treatment_id.rename(columns={'Number_By_Appt_Type_Code': 'Number_of_HSI'}, inplace=True)

# Find the appt footprint for each treatment_id
appts_by_treatment_id = \
    hsi.set_index('TREATMENT_ID')['Number_By_Appt_Type_Code'].drop_duplicates().apply(pd.Series).fillna(0.0)
# todo: follow the change in analysis_hsi_in_typical_run

Results folder is: C:\Users\jdbb1\Desktop\TLOmodel\outputs\bshe@ic.ac.uk\scenario_hsi_in_typical_run-2021-12-10T180225Z


In [4]:
# The Sankey

# Prepare the data: appt type and number per hsi
appt_and_hsi = appts_by_treatment_id.reset_index().copy()
appt_and_hsi = pd.melt(appt_and_hsi, id_vars=['TREATMENT_ID'], value_vars=appt_and_hsi.columns[1:],
                      var_name='Appt_Type')
appt_and_hsi = appt_and_hsi[appt_and_hsi['value'] > 0].copy()

# todo:
# Prepare the data plus: total number of appts per hsi for year 2010-2018
# num_appt_by_hsi = appts_by_treatment_id.copy()
# for event in num_appt_by_hsi.index:
#     num_appt_by_hsi.loc[event,:] = appts_by_treatment_id.loc[event,:] * num_hsi_by_treatment_id.loc[event, 'Number_of_HSI']
# num_appt_by_hsi = num_appt_by_hsi.reset_index().copy()
# num_appt_by_hsi = pd.melt(num_appt_by_hsi, id_vars=['TREATMENT_ID'], value_vars=num_appt_by_hsi.columns[1:],
#                       var_name='Appt_Type')
# num_appt_by_hsi = num_appt_by_hsi[num_appt_by_hsi['value'] > 0].copy()

# Define the flow
appt_and_hsi['source'] = 'Appt_Type'
appt_and_hsi['target'] = 'TREATMENT_ID'

size = dict(width=1000, height=800, margins=dict(left=120, right=520))

partition_appt_type = Partition.Simple('Appt_Type', np.unique(appt_and_hsi['Appt_Type']))

partition_treatment_id = Partition.Simple('TREATMENT_ID', np.unique(appt_and_hsi['TREATMENT_ID']))

nodes = {
    'Appt': ProcessGroup(['Appt_Type'], partition_appt_type),
    'HSI': ProcessGroup(['TREATMENT_ID'], partition_treatment_id),
}

bundles = [
    Bundle('Appt', 'HSI'),
]

ordering = [
    ['Appt'],  # left
    ['HSI'],   # right
    ]

# Set the color for each appt type
palette = {'IPAdmission': 'lightsteelblue', 'InpatientDays': 'skyblue',
           'Over5OPD': 'cornflowerblue', 'Under5OPD': 'steelblue',
           'AntenatalFirst': 'plum', 'ANCSubsequent': 'hotpink',
           'CompDelivery': 'tomato', 'NormalDelivery': 'darksalmon',
           'FamPlan': 'gold', 'MajorSurg': 'orange', 'ConWithDCSA': 'mediumpurple',
           'MaleCirc': 'lightgreen', 'NewAdult': 'mediumseagreen',
           'VCTNegative': 'greenyellow', 'VCTPositive': 'olivedrab',
          }

# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_appt_type)

# Generate and save Sankey
sankey_appt_and_hsi = weave(sdd, appt_and_hsi, palette=palette, measures='value').to_widget(**size)

sankey_appt_and_hsi.auto_save_png(make_graph_file_name('Sankey_appt_and_hsi'))